In [1]:
from torchvision import datasets

/home/fsamir/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/fsamir/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
%%writefile preprocess.py

import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer

root_dir = '../input/caltech256/256_ObjectCategories'
# get all the folder paths
all_paths = os.listdir(root_dir)

# create a DataFrame
data = pd.DataFrame()

images = []
labels = []
counter = 0
for folder_path in tqdm(all_paths, total=len(all_paths)):
    # get all the image names in the particular folder
    image_paths = os.listdir(f"{root_dir}/{folder_path}")
    # get the folder as label
    label = folder_path.split('.')[-1]
    
    if label == 'clutter':
        continue

    # save image paths in the DataFrame
    for image_path in image_paths:
        if image_path.split('.')[-1] == 'jpg':
            data.loc[counter, 'image_path'] = f"{root_dir}/{folder_path}/{image_path}"
            labels.append(label)
            counter += 1

labels = np.array(labels)
# one-hot encode the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# add the image labels to the dataframe
for i in range(len(labels)):
    index = np.argmax(labels[i])
    data.loc[i, 'target'] = int(index)
    
# shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True)

print(f"Number of labels or classes: {len(lb.classes_)}")
print(f"The first one hot encoded labels: {labels[0]}")
print(f"Mapping the first one hot encoded label to its category: {lb.classes_[0]}")
print(f"Total instances: {len(data)}")
 
# save as CSV file
data.to_csv('data.csv', index=False)
print(data.head(5))

Overwriting preprocess.py


In [48]:
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
def get_labels():
    labels = []
    root_dir = 'data/256_ObjectCategories'
# get all the folder paths
    all_paths = os.listdir(root_dir)
    for folder_path in tqdm(all_paths, total=len(all_paths)):
    # get all the image names in the particular folder
        image_paths = os.listdir(f"{root_dir}/{folder_path}")
        # get the folder as label
        label = folder_path.split('.')[-1]
        
        if label == 'clutter':
            continue

        # save image paths in the DataFrame
        for _ in image_paths:
            labels.append(label)
    return labels
        
def get_mapping_label_ind_to_concept(labels):
    labels = np.array(labels)
# one-hot encode the labels
    lb = LabelBinarizer()
    labels_binarized = lb.fit_transform(labels)

    # add the image labels_binarized to the dataframe
    label_ind_to_concept_map = {}
    for i in range(len(labels_binarized)):
        index = np.argmax(labels_binarized[i])
        label_ind_to_concept_map[index] = labels[i]
    return label_ind_to_concept_map

label_ind_to_concept_map = get_mapping_label_ind_to_concept(get_labels())



100%|██████████| 258/258 [00:00<00:00, 22363.15it/s]


In [49]:
print(label_ind_to_concept_map)

{97: 'guitar-pick', 22: 'buddha-101', 174: 'refrigerator', 250: 'windmill', 71: 'fighter-jet', 84: 'galaxy', 253: 'yarmulke', 118: 'ice-cream-cone', 79: 'french-horn', 25: 'cactus', 144: 'microscope', 13: 'birdbath', 205: 'steering-wheel', 117: 'ibis-101', 182: 'scorpion-101', 146: 'minaret', 177: 'rotary-phone', 197: 'socks', 88: 'golden-gate-bridge', 237: 'tripod', 233: 'treadmill', 162: 'people', 81: 'frisbee', 115: 'human-skeleton', 207: 'sunflower-101', 210: 'swan', 78: 'football-helmet', 255: 'zebra', 244: 'washing-machine', 220: 'tennis-ball', 116: 'hummingbird', 95: 'grasshopper', 49: 'conch', 228: 'tombstone', 72: 'fire-extinguisher', 145: 'microwave', 215: 'tambourine', 201: 'spider', 136: 'lightning', 43: 'coffin', 101: 'harp', 165: 'picnic-table', 124: 'kangaroo-101', 65: 'elephant-101', 246: 'waterfall', 137: 'llama-101', 185: 'self-propelled-lawn-mower', 231: 'tower-pisa', 44: 'coin', 130: 'laptop-101', 122: 'jesus-christ', 209: 'sushi', 8: 'bathtub', 67: 'ewer-101', 196:

In [3]:
%%writefile dataset.py

import albumentations
import numpy as np
import torch

from PIL import Image
from torch.utils.data import Dataset

# custom dataset
class ImageDataset(Dataset):
    def __init__(self, images, labels=None, tfms=None):
        self.X = images
        self.y = labels

        # apply augmentations
        if tfms == 0: # if validating
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
            ])
        else: # if training
            self.aug = albumentations.Compose([
                albumentations.Resize(224, 224, always_apply=True),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.3,
                    scale_limit=0.3,
                    rotate_limit=15,
                    p=0.5
                ),
            ])
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        image = Image.open(self.X[i])
        image = image.convert('RGB')
        image = self.aug(image=np.array(image))['image']
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        label = self.y[i]
        return {
            'image': torch.tensor(image, dtype=torch.float), 
            'target': torch.tensor(label, dtype=torch.long)
        }

Overwriting dataset.py


In [4]:
%%writefile model.py

import pretrainedmodels
import torch.nn as nn
import torch.nn.functional as F

class ResNet50(nn.Module):
    def __init__(self, pretrained, requires_grad):
        super(ResNet50, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__['resnet50'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet50'](pretrained=None)
            
        if requires_grad == True:
            for param in self.model.parameters():
                param.requires_grad = True
        elif requires_grad == False:
            for param in self.model.parameters():
                param.requires_grad = False
        
        self.l0 = nn.Linear(2048, 256)

    def forward(self, x):
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        l0 = self.l0(x)
        return l0

model = ResNet50(pretrained=True, requires_grad=False)
# print(model)

Overwriting model.py


In [5]:
from model import model
from dataset import ImageDataset

In [6]:
import pandas as pd 
from torch.utils.data import DataLoader
import torch

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# get the dataset ready
df = pd.read_csv('data.csv')
X = df.image_path.values # image paths
y = df.target.values # targets

In [10]:
image_data = ImageDataset(X, y, tfms=0)


In [28]:
from typing import Dict, Iterable


def collate_imgs(img_dicts: Iterable[Dict]):
    img_tensors = [img_dict['image'] for img_dict in img_dicts]
    label_tensors = [img_dict['target'] for img_dict in img_dicts] 

    batch_image_tensor = torch.stack(img_tensors)
    batch_label_tensor = torch.stack(label_tensors)

    return batch_image_tensor, batch_label_tensor



In [53]:
dataloader = DataLoader(image_data, 128, shuffle=True, collate_fn=collate_imgs )
image_iter = iter(dataloader)

In [51]:
model = model.cuda()

In [54]:

feats = []
labels = []
for batch_X, batch_y in iter(dataloader):
    feats.append(model(batch_X.cuda()).cpu())
    labels.append(batch_y)


In [58]:
all_feats = torch.cat(feats, dim=0)
all_labels = torch.cat(labels, dim=0)

In [61]:
all_labels_text = [label_ind_to_concept_map[label.item()] for label in all_labels]


In [64]:
torch.save(all_feats, 'results/all_feats.pt')
torch.save(all_labels, 'results/all_labels.pt')

In [63]:
import pickle
with open("results/all_labels_text.pkl", "wb") as pkl_f:
    pickle.dump(all_labels_text, pkl_f, protocol=pickle.HIGHEST_PROTOCOL)